In [19]:
import pytest
import sys

# Pytest Best Practice

This module covers organising your test suite and some interesting tricks around test
suite execution.

## Test Suite Organisation

The test folder should mirror the structure of the src code folder. Use the same package
names etc.

Segregation of tests for functions and classes should happen in test classes. For
example:  
`tree some_package`
```
/some_package
├── src
│   ├── foo
│   │   ├── foobar1.py
│   │   └── foobar2.py
│   └── bar
│       ├── barfoo1.py
│       └── barfoo2.py
└── tests
    ├── foo
    │   ├── test_foobar1.py
    │   └── test_foobar2.py
    └── bar
        ├── test_barfoo1.py
        └── test_barfoo2.py



```
Then let's take a look at the organisation within `tests/foo/test_foobar1.py` 

The functions to test within that module are defined below.

In [7]:
def fizz(num):
    """return fizz if num is cleanly divisible by 3

    Args:
        num (int): an integer.
    """
    if num % 3 == 0:
        return "fizz"
    else:
        return num


def buzz(num):
    """return buzz if num is cleanly divisible by 5

    Args:
        num (int): an integer
    """
    if num % 5 == 0:
        return "buzz"
    else:
        return num

Now in the test suite, content of `tests/foo/test_foobar1.py`:

In [16]:
class TestFizz(object):
    def test_fizz_fizzes(self):
        results = list()
        ints = [3, 30, 300]
        for i in ints:
            results.append(fizz(i))
        assert all([f == "fizz" for f in results])


class TestBuzz(object):
    def test_buzz_buzzes(self):
        results = list()
        ints = [5, 50, 500]
        for i in ints:
            results.append(buzz(i))
        assert all([f == "buzz" for f in results])

## Test Suite Execution

Some neat tricks to run from the cli for targetted test execution:

Run `pytest` to run everything.  

Run `pytest -x` to run the suite until the first fail is encountered.

To run a specific package test suite:  
`pytest foo`

To run a specific test module:  
`pytest foo/test_foobar1.py`

To run a specific test class, use its node ID:  
`pytest foo/test_foobar1.py::TestFizz`

You can also use the `-k` (keyword flag) to use rules to identify your tests of interest
. Be sure to include a quoted string. This is much quicker:
`pytest -k "TestFizz"`

You can also use python syntax to be even more specific:  
`pytest -k "TestFizz and not some_unwanted_test"`

## Marking Fails

Some tests may be expected to fail. These can be silenced with some decorators that
make this explicit.


In [18]:
@pytest.mark.xfail(reason="Not yet implemented")
class TestFizzbuzz(object):
    def test_fizzbuzz_fizzbuzzes(self):
        res = fizzbuzz(15)
        assert res == "fizzbuzz"

Running pytest will not report a failure. A good idea to include the following
parameters to ensure the reason is implemented:

`pytest -r`  
This includes reasons. Note, you must **also include the s or x option, see below.**

`pytest -rx`  
This includes reasons for xfail'ed tests.

## Marking Skips

If you would like to skip tests on certain platforms or python versions, use the skipif
decorator.

In [20]:
# a test that uses deprecated print statement
@pytest.mark.skipif(sys.version_info >= (3, 0), reason="Only works with python 2")
def test_uses_python2_print():
    print "Old python syntax"
    pass

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (3965558869.py, line 4)

While this causes an error in notebook, it should silently pass in a pytest suite run.
Again, print reason with the -r flag. Be specific for skips with:  
`pytest -rs`

## Continuous Integration

The module finishes with a light touch consideration of CI with Travis & Codecov. The
Codecov implementation appears to be considerably simpler than that of GitHub Actions but also appears
to be a bit outdated. I include the YAML represented in the course that enables test
suite execution, coverage report generation and upload to codecov.io:

in project root `.travis.yaml`:

```
language: python
python:
  - "3.7"
install:
  - pip install -e .
  - pip install pytest-cov codecov # dep for codecov
script:
  - pytest --cov=src tests # need to point to package src
after_success:
  - codecov # travis will push report to codecov.io

```

Note the above assumes that travis & codecov have been given access to the repo.
